In [2]:
import damask
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

/home/snkrkr/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/snkrkr/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/snkrkr/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [6]:
resultfile_path = 'reference_output/tensionX_tensionY_material_meanfield_20materials_0.5ferrite_0.5martensite_20grains_voronoi_4x4x4/simulation_output.hdf5'

In [7]:
result = damask.Result(resultfile_path)
try:
    result.add_stress_Cauchy()
except ValueError:
    pass
try:
    result.add_equivalent_Mises('sigma')
except ValueError:
    pass
try:
    result.add_stress_second_Piola_Kirchhoff()
except ValueError:
    pass

try:
    result.add_strain()
except ValueError:
    pass

postprossecing_dir = Path(resultfile_path).stem
vti_path = list(Path(resultfile_path).parent.glob("*.vti"))[0]
grid = damask.GeomGrid.load(vti_path)
grid

cells:  2 × 5 × 1
size:   2e-05 × 5e-05 × 1e-05 m³
origin: 0.0   0.0   0.0 m
# materials: 10

In [8]:
for inc in result.get(['F','P']).values():
    print(inc.keys())
    break

dict_keys(['Ferrite', 'Martensite'])


# Example: Stress-strain curve for tensionX

In [9]:
def plot_stress_strain_per_grain_meanfield(grains, grid, result):
    data = {g:pd.DataFrame() for g in grains}
    for volumefractions in result.get(['F','P']).values():
        for volumefraction, inc in volumefractions.items():
            P = inc['P']
            F = inc['F']
            for g in grains:
                print(volumefraction, g)
                points = grid.material.flatten(order='F')==g
                flattened_material = grid.material.flatten(order='F')
                print("flattened_material.shape", flattened_material.shape)

                print("grid.material.shape", grid.material.shape)
                print("P.shape", P.shape)
                print("P", P)
                P_11 = P[points,0,0].flatten()
                F_11 = np.broadcast_to(np.average(F[:,0,0]),P_11.shape)
                x = pd.DataFrame({'F_11':F_11,'P_11':P_11})
                data[g] = pd.concat((data[g],x),ignore_index=True)

plot_stress_strain_per_grain_meanfield([0, 1, 2], grid, result)


Ferrite 0
flattened_material.shape (10,)
grid.material.shape (2, 5, 1)
P.shape (64, 3, 3)
P [[[-1.61040548e-04 -6.40257459e-06  6.43587931e-06]
  [-6.40257459e-06 -1.75632446e-04 -1.33571067e-05]
  [ 6.43587931e-06 -1.33571067e-05 -1.95956502e-04]]

 [[-1.66034953e-04 -6.77649153e-06 -2.30301635e-05]
  [-6.77649153e-06 -1.05469715e-04  5.04107563e-06]
  [-2.30301635e-05  5.04107563e-06 -6.48929088e-05]]

 [[ 1.95955715e-04 -3.48692999e-06  7.71512225e-06]
  [-3.48692999e-06  2.01962088e-04 -2.16109212e-06]
  [ 7.71512225e-06 -2.16109212e-06  2.18811087e-04]]

 [[-1.11326587e-04 -4.75446784e-06 -2.20671113e-05]
  [-4.75446784e-06 -1.01048161e-04  1.39961463e-05]
  [-2.20671113e-05  1.39961463e-05 -1.80089092e-04]]

 [[-2.72071621e-05 -1.88653822e-05 -4.63475067e-05]
  [-1.88653822e-05 -9.96843402e-06  1.61816687e-05]
  [-4.63475067e-05  1.61816687e-05  3.71755961e-05]]

 [[-2.72071621e-05 -1.88653822e-05 -4.63475067e-05]
  [-1.88653822e-05 -9.96843402e-06  1.61816687e-05]
  [-4.63475067

IndexError: boolean index did not match indexed array along dimension 0; dimension is 64 but corresponding boolean dimension is 10

# Example: Stress-Strain Density plot for each grain 

In [17]:
result = damask.Result(resultfile_path)
last_inc = result.view(increments=-1)
last_inc = result.view(homogenizations=False)
last_inc
df = pd.DataFrame({'F_11':last_inc.get('F')[:,0,0],
                   'P_11':last_inc.get('P')[:,0,0]})
# _ = df.plot.hexbin(x='F_11',y='P_11',gridsize=20,
#                    cmap=damask.Colormap.from_predefined('strain'))

TypeError: unhashable type: 'slice'

# Example: r-value calculation

In [1]:
result = damask.Result(resultfile_name)
result.add_strain('F_p','U')
result.add_equivalent_Mises('epsilon_U^0.0(F_p)')
epsilon_avg = np.array([np.average(eps,0) for eps in result.get('epsilon_U^0.0(F_p)').values()])
r = epsilon_avg[:,1,1]/epsilon_avg[:,2,2]
s = np.array([np.average(strain) for strain in result.get('epsilon_U^0.0(F_p)_vM').values()])
plt.plot(s[10:],r[10:])
plt.ylabel('r-value')
plt.xlabel('plastic vM strain')
plt.axis([0, 0.2, 0, 1])
plt.show()